<a href="https://colab.research.google.com/github/rroszczyk/Python/blob/master/szyfry/rsa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math

In [2]:
msg = 19
 
p = 3
q = 7
e = 2
 
n = p * q  ## klucz publiczny n oraz e
 
phi = (p - 1) * (q - 1)

while (e < phi):
  if (math.gcd(e, phi) == 1):
    break
  else:
    e = e + 1

k = 2
d = int((1 + (k * phi)) / e)  ## klucz prywatny d oraz n

print(f"klucz publiczny: {n}, {e}")
print(f"klucz prywanty: {d}, {n}")

def encrypt(msg):
    en = math.pow(msg, e)
    c = int(en % n)
    print("Zakodowana wiadomość: ", c)
    return c

def decrypt(me):
  de = math.pow(me, d)
  c = int(de % n)
  print("Odkodowana wiadomość", c)
  return c

print("Wiadomość oryginalna: ", msg)
c = encrypt(msg)
c = decrypt(c)

klucz publiczny: 21, 5
klucz prywanty: 5, 21
Wiadomość oryginalna:  19
Zakodowana wiadomość:  10
Odkodowana wiadomość 19


In [3]:
!pip install pycryptodome
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
from Crypto.Signature.pkcs1_15 import PKCS115_SigScheme
from Crypto.Hash import SHA256
import binascii

     |████████████████████████████████| 2.0 MB 9.6 MB/s 


In [4]:
keyPair = RSA.generate(1024)

pubKey = keyPair.publickey()
print(f"Public key:  (n={hex(pubKey.n)}, e={hex(pubKey.e)})")
pubKeyPEM = pubKey.exportKey()
print(pubKeyPEM.decode('ascii'))

print(f"Private key: (n={hex(pubKey.n)}, d={hex(keyPair.d)})")
privKeyPEM = keyPair.exportKey()
print(privKeyPEM.decode('ascii'))

Public key:  (n=0xe23b4ba8e01c42364b9b9b98c3d023c33d2da5ccdba04d22cbdd54d16f30236081390818d67f322b9b30fc46d97d2165f60a7dc470f70bd4e9bcea7cae6974d6aa35ebca968b6af4769aa63b8275808b76e0d4485c39f680f02e87af49b4d741283a123e7ae2653c5d50e8d25b1b0655475d456faebce63f9bab2e9f5ac6062b, e=0x10001)
-----BEGIN PUBLIC KEY-----
MIGfMA0GCSqGSIb3DQEBAQUAA4GNADCBiQKBgQDiO0uo4BxCNkubm5jD0CPDPS2l
zNugTSLL3VTRbzAjYIE5CBjWfzIrmzD8Rtl9IWX2Cn3EcPcL1Om86nyuaXTWqjXr
ypaLavR2mqY7gnWAi3bg1EhcOfaA8C6Hr0m010EoOhI+euJlPF1Q6NJbGwZVR11F
b6685j+bqy6fWsYGKwIDAQAB
-----END PUBLIC KEY-----
Private key: (n=0xe23b4ba8e01c42364b9b9b98c3d023c33d2da5ccdba04d22cbdd54d16f30236081390818d67f322b9b30fc46d97d2165f60a7dc470f70bd4e9bcea7cae6974d6aa35ebca968b6af4769aa63b8275808b76e0d4485c39f680f02e87af49b4d741283a123e7ae2653c5d50e8d25b1b0655475d456faebce63f9bab2e9f5ac6062b, d=0x185af8dcbfefdae2fc85ce9d062bb0c5f29ea8f4628aa1f3468e57f4ca2b1d0be52d8e251a668c30ac9efbf1e6b53574c425c1ff7587eb32ab4d46d3507592f9db7ce7b0073b442107e5fe10cf28d7272

In [5]:
msg = b'tekst do zakodowania'
encryptor = PKCS1_OAEP.new(pubKey)
encrypted = encryptor.encrypt(msg)
print("Zakodowany:", binascii.hexlify(encrypted))

Zakodowany: b'3e6789f11d0030841a1fff89b938dde0ddd4a29f536185a049a71aac41e515de095283c53a8ce56a85019e6fb2050c4f71f8e7d5d3baf70cda07ca32ff29e7649e9f962aaba903e719f05a1b2c25a997308bcb6e9b40c653ee0e01d9e8d83dfeb4f6adf550fe07682ca1e6681b4ef99da580c12d70fdecda97b87349cd3b98cf'


In [6]:
decryptor = PKCS1_OAEP.new(keyPair)
decrypted = decryptor.decrypt(encrypted)
print('Odkodowany:', decrypted)

Odkodowany: b'tekst do zakodowania'


In [7]:
msg = b'tekst do zakodowania'
hash = SHA256.new(msg)
signer = PKCS115_SigScheme(keyPair)
signature = signer.sign(hash)
print("Podpis:", binascii.hexlify(signature))

msg = b'tekst do zakodowania'
hash = SHA256.new(msg)
signer = PKCS115_SigScheme(keyPair)
try:
    signer.verify(hash, signature)
    print("Podpis jest poprawny")
except:
    print("Podpis jest błędny")

msg = b'podrobiony tekst do zakodowania'
hash = SHA256.new(msg)
signer = PKCS115_SigScheme(keyPair)
try:
    signer.verify(hash, signature)
    print("Podpis jest poprawny")
except:
    print("Podpis jest błędny")

Podpis: b'0abf73c801bd3bf44364e2321099b882be4923748152f07faf23fc324d92dde677a51768df9cd40fddba9266d7e62609f076c0591fab087a354abcca0d8b7b4d7b3c2a3898006881cceac4d49d789bda103eeefb40f8bbce7e81e6b652fcb0bcbe10ab5bc3d2f08d45592e1ffb7469d7f92122f9386df12bde4c69c1d10da0e4'
Podpis jest poprawny
Podpis jest błędny
